# Stage 1: Importing dependencies

In [ ]:
!pip install sentencepiece
!pip install tf-models-official
#!pip install tf-models-nightly # better to install the version in development
!pip install tf-nightly

     |████████████████████████████████| 1.2 MB 14.0 MB/s 
     |████████████████████████████████| 2.2 MB 14.9 MB/s 
     |████████████████████████████████| 4.9 MB 74.8 MB/s 
     |████████████████████████████████| 90 kB 13.0 MB/s 
     |████████████████████████████████| 352 kB 71.8 MB/s 
     |████████████████████████████████| 99 kB 12.0 MB/s 
     |████████████████████████████████| 636 kB 62.9 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
     |████████████████████████████████| 234 kB 78.9 MB/s 
     |████████████████████████████████| 497.5 MB 25 kB/s 
     |████████████████████████████████| 47.7 MB 1.4 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 15.7 MB 52.4 MB/s 
     |████████████████████████████████| 462 kB 69.1 MB/s 
     |████████████████████████████████| 1.4 MB 58.9 MB/s 
     |████████████████████████████████| 5.8 MB 55.2 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl 

     |███████████████████████▌        | 365.7 MB 39.1 MB/s eta 0:00:04

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.8.0'

In [3]:
import tensorflow_hub as hub

from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file

from official.nlp import optimization

from official.nlp.data.squad_lib import read_squad_examples
from official.nlp.data.squad_lib import FeatureWriter
from official.nlp.data.squad_lib import convert_examples_to_features
from official.nlp.data.squad_lib import write_predictions

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [4]:
import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive

# Stage 2: Data preprocessing

In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/MyDrive/Datasets/Squad_1.0/train-v1.1.json",
    "/content/drive/MyDrive/Datasets/Squad_1.0/vocab.txt",
    "/content/drive/MyDrive/Datasets/Squad_1.0/train-v1.1.tf_record")

In [34]:
import pprint
pp = pprint.PrettyPrinter(depth=3)

with open("/content/drive/MyDrive/Datasets/Squad_1.0/train-v1.1.json") as f:
  data = json.load(f)['data']
  num_topics = len(data)
  print(f"#Topics: {num_topics}")

  total_training_ex = 0
  print_ex = True

  for topic_idx in range(num_topics):
    topic = data[topic_idx]
    title = topic['title']
    paras = topic['paragraphs']
    num_paras = len(paras)
    
    if print_ex:
      print(f"[{topic_idx}] Topic Title : {title}")
      print(f"#Paragraphs : {num_paras}")
    
    for para_idx in range(num_paras):
      para = paras[para_idx]
      qas = para['qas']
      num_ques = len(qas)

      if print_ex:
        print(f"[{para_idx}] Context : {para['context']}")

      for ques_idx in range(len(qas)):
        ques = qas[ques_idx]['question']
        ans = qas[ques_idx]['answers']

        total_training_ex += len(ans)

        if print_ex:
          print(f"Q{ques_idx + 1} : {ques}")
          print(f"A{ques_idx + 1} : {ans}")

        #pp.pprint(qas)
      print_ex = False
    
  
  print(f"No of training examples : {total_training_ex}")


#Topics: 442
[0] Topic Title : University_of_Notre_Dame
#Paragraphs : 55
[0] Context : Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Q1 : To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
A1 : [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]
Q2 : What is in front of the Notre Dame Main Building?
A2 : [{'answer_sta

In [7]:
with tf.io.gfile.GFile("/content/drive/MyDrive/Datasets/Squad_1.0/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [7]:
with tf.io.gfile.GFile("/content/drive/MyDrive/Datasets/Squad_1.0/train_meta_data", "r") as r:
    input_meta_data = json.load(r)

In [8]:
print(json.dumps(input_meta_data, indent=4))

{
    "task_type": "bert_squad",
    "train_data_size": 88641,
    "max_seq_length": 384,
    "max_query_length": 64,
    "doc_stride": 128,
    "version_2_with_negative": false
}


In [9]:
BATCH_SIZE = 4

train_dataset = create_squad_dataset(
    "/content/drive/MyDrive/Datasets/Squad_1.0/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training=True)

# Stage 3: Model building

## Squad layer

In [10]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
        units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2)

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)] 
    return unstacked_logits[0], unstacked_logits[1]

## Whole model

In [11]:
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer()
    
    def apply_bert(self, inputs):
#        _ , sequence_output = self.bert_layer([inputs["input_ids"],
#                                               inputs["input_mask"],
#                                               inputs["segment_ids"]])
        
        # New names for the 3 different elements of the inputs, since an update
        # in tf_models_officials. Doesn't change anything for any other BERT
        # usage.
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],
                                               inputs["input_mask"],
                                               inputs["input_type_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits

# Stage 4: Training

## Creating the AI

In [12]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 2000
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [13]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN)

In [14]:
bert_squad = BERTSquad()

In [15]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS)

In [16]:
def squad_loss_fn(labels, model_outputs):
    start_positions = labels['start_positions']
    end_positions = labels['end_positions']
    start_logits, end_logits = model_outputs

    start_loss = tf.keras.backend.sparse_categorical_crossentropy(
        start_positions, start_logits, from_logits=True)
    end_loss = tf.keras.backend.sparse_categorical_crossentropy(
        end_positions, end_logits, from_logits=True)
    
    total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

    return total_loss

train_loss = tf.keras.metrics.Mean(name="train_loss")

In [17]:
next(iter(train_dataset_light))

({'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'input_word_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[ 101, 2105, 2054, ...,    0,    0,    0],
         [ 101, 2029, 2533, ...,    0,    0,    0],
         [ 101, 2129, 2116, ...,    0,    0,    0],
         [ 101, 2054, 7017, ...,    0,    0,    0]], dtype=int32)>},
 {'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([202, 172,  32, 191], dtype=int32)>,
  'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([201, 167,  29, 188], dtype=int32)>})

In [18]:
bert_squad.compile(optimizer,
                   squad_loss_fn)

In [19]:
checkpoint_path = "/content/drive/MyDrive/BERT_UDEMY/models/ckpt_bert_squad/"

ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


## Custom training

In [20]:
!nvidia-smi

Sat Feb 12 20:06:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W /  70W |   2342MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
for epoch in range(NB_EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    
    for (batch, (inputs, targets)) in enumerate(train_dataset_light):
        with tf.GradientTape() as tape:
            model_outputs = bert_squad(inputs)
            loss = squad_loss_fn(targets, model_outputs)
        
        gradients = tape.gradient(loss, bert_squad.trainable_variables)
        optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))
        
        train_loss(loss)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f}".format(
                epoch+1, batch, train_loss.result()))
        
        if batch % 500 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                                ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

Start of epoch 1
Epoch 1 Batch 0 Loss 5.9036
Saving checkpoint for epoch 1 at /content/drive/MyDrive/BERT_UDEMY/models/ckpt_bert_squad/ckpt-2
Epoch 1 Batch 50 Loss 5.7067
Epoch 1 Batch 100 Loss 5.1587
Epoch 1 Batch 150 Loss 4.4845
Epoch 1 Batch 200 Loss 4.0156
Epoch 1 Batch 250 Loss 3.6229
Epoch 1 Batch 300 Loss 3.3475
Epoch 1 Batch 350 Loss 3.1951
Epoch 1 Batch 400 Loss 3.0206
Epoch 1 Batch 450 Loss 2.8601
Epoch 1 Batch 500 Loss 2.7166
Saving checkpoint for epoch 1 at /content/drive/MyDrive/BERT_UDEMY/models/ckpt_bert_squad/ckpt-3
Epoch 1 Batch 550 Loss 2.5959
Epoch 1 Batch 600 Loss 2.5263
Epoch 1 Batch 650 Loss 2.4543
Epoch 1 Batch 700 Loss 2.3852
Epoch 1 Batch 750 Loss 2.3227
Epoch 1 Batch 800 Loss 2.2554
Epoch 1 Batch 850 Loss 2.2245
Epoch 1 Batch 900 Loss 2.1821
Epoch 1 Batch 950 Loss 2.1351
Epoch 1 Batch 1000 Loss 2.0768
Saving checkpoint for epoch 1 at /content/drive/MyDrive/BERT_UDEMY/models/ckpt_bert_squad/ckpt-4
Epoch 1 Batch 1050 Loss 2.0312
Epoch 1 Batch 1100 Loss 1.9758
Ep

# Stage 5: Evaluation

## Prepare evaluation

Get the dev set in the session

In [22]:
eval_examples = read_squad_examples(
    "/content/drive/MyDrive/Datasets/Squad_1.0/dev-v1.1.json",
    is_training=False,
    version_2_with_negative=False)

Define the function that will write the tf_record file for the dev set

In [23]:
eval_writer = FeatureWriter(
    filename=os.path.join("/content/drive/MyDrive/Datasets/Squad_1.0/",
                          "eval.tf_record"),
    is_training=False)

Create a tokenizer for future information needs

In [24]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

Define the function that add the features (feature is a protocol in tensorflow) to our eval_features list

In [25]:
def _append_feature(feature, is_padding):
    if not is_padding:
        eval_features.append(feature)
    eval_writer.process_feature(feature)

Create the eval features and the writes the tf.record file

In [26]:
eval_features = []
dataset_size = convert_examples_to_features(
    examples=eval_examples,
    tokenizer=tokenizer,
    max_seq_length=384,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    output_fn=_append_feature,
    batch_size=4)

In [27]:
eval_writer.close()

Load the ready-to-be-used dataset to our session

In [28]:
BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/drive/MyDrive/Datasets/Squad_1.0/eval.tf_record",
    384,#input_meta_data['max_seq_length'],
    BATCH_SIZE,
    is_training=False)

## Making the predictions

Defines a certain type of collection (like a dictionary)

In [29]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

Returns each element of batched output at a time

In [30]:
def get_raw_results(predictions):
    for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                    predictions['start_logits'],
                                                    predictions['end_logits']):
        yield RawResult(
            unique_id=unique_ids.numpy(),
            start_logits=start_logits.numpy().tolist(),
            end_logits=end_logits.numpy().tolist())

Let's make our predictions!

In [31]:
all_results = []
for count, inputs in enumerate(eval_dataset):
    x, _ = inputs
    unique_ids = x.pop("unique_ids")
    start_logits, end_logits = bert_squad(x, training=False)
    output_dict = dict(
        unique_ids=unique_ids,
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict):
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

0/2709
100/2709
200/2709
300/2709
400/2709
500/2709
600/2709
700/2709
800/2709
900/2709
1000/2709
1100/2709
1200/2709
1300/2709
1400/2709
1500/2709
1600/2709
1700/2709
1800/2709
1900/2709
2000/2709
2100/2709
2200/2709
2300/2709
2400/2709
2500/2709
2600/2709
2700/2709


Write the predictions in a json file that will work with the evaluation script

In [32]:
output_prediction_file = "/content/drive/MyDrive/Datasets/Squad_1.0/predictions.json"
output_nbest_file = "/content/drive/MyDrive/Datasets/Squad_1.0/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/MyDrive/Datasets/Squad_1.0/null_odds.json"

write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)

## Home-made prediction

### Input dict creation

We will concatenate the question and the context, separated by a `["SEP"]`, after tokenization, as it has been made for the training set.

The important thing is that we want our answer to start with a real word and to end with a real word. The word "ecologically" being tokenized as `["ecological", "##ly"]`, if the ending token is `["ecological"]` we want to use "ecologically" as the ending word (same thing if the ending token is `["##ly"]`). That is why we first split our context into words, and then into tokens, remembering to which word each token belongs to (see the `tokenize_context()` function).

#### Utils

In [33]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)
vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [34]:
def is_whitespace(c):
    '''
    Tell if a chain of characters corresponds to a whitespace or not.
    '''
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [35]:
def whitespace_split(text):
    '''
    Take a text and return a list of "words" by splitting it according to
    whitespaces.
    '''
    doc_tokens = []
    prev_is_whitespace = True
    for c in text:
        if is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
    return doc_tokens

In [36]:
def tokenize_context(text_words):
    '''
    Take a list of words (returned by whitespace_split()) and tokenize each word
    one by one. Also keep track, for each new token, of its original word in the
    text_words parameter.
    '''
    text_tok = []
    tok_to_word_id = []
    for word_id, word in enumerate(text_words):
        word_tok = tokenizer.tokenize(word)
        text_tok += word_tok
        tok_to_word_id += [word_id]*len(word_tok)
    return text_tok, tok_to_word_id

In [37]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids

In [38]:
def create_input_dict(question, context):
    '''
    Take a question and a context as strings and return a dictionary with the 3
    elements needed for the model. Also return the context_words, the
    context_tok to context_word ids correspondance and the length of
    question_tok that we will need later.
    '''
    question_tok = tokenizer.tokenize(my_question)

    context_words = whitespace_split(context)
    context_tok, context_tok_to_word_id = tokenize_context(context_words)

    input_tok = question_tok + ["[SEP]"] + context_tok + ["[SEP]"]
    input_tok += ["[PAD]"]*(384-len(input_tok)) # in our case the model has been
                                                # trained to have inputs of length max 384
    input_dict = {}
    input_dict["input_word_ids"] = tf.expand_dims(tf.cast(get_ids(input_tok), tf.int32), 0)
    input_dict["input_mask"] = tf.expand_dims(tf.cast(get_mask(input_tok), tf.int32), 0)
    input_dict["input_type_ids"] = tf.expand_dims(tf.cast(get_segments(input_tok), tf.int32), 0)

    return input_dict, context_words, context_tok_to_word_id, len(question_tok)

#### Creation

In [39]:
my_context = '''Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.'''

Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.

In [40]:
#my_question = '''What philosophy of thought addresses wealth inequality?'''
my_question = '''What are examples of economic actors?'''
#my_question = '''In a market economy, what is inequality a reflection of?'''

In [42]:
my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, my_context)

### Prediction

In [43]:
start_logits, end_logits = bert_squad(my_input_dict, training=False)

### Interpretation

We remove the ids corresponding to the question and the `["SEP"]` token:

In [44]:
start_logits_context = start_logits.numpy()[0, question_tok_len+1:]
end_logits_context = end_logits.numpy()[0, question_tok_len+1:]

First easy interpretation:

In [45]:
start_word_id = context_tok_to_word_id[np.argmax(start_logits_context)]
end_word_id = context_tok_to_word_id[np.argmax(end_logits_context)]

"Advanced" - making sure that the start of the answer is before the end:

In [46]:
pair_scores = np.ones((len(start_logits_context), len(end_logits_context)))*(-1E10)
for i in range(len(start_logits_context-1)):
    for j in range(i, len(end_logits_context)):
        pair_scores[i, j] = start_logits_context[i] + end_logits_context[j]
pair_scores_argmax = np.argmax(pair_scores)

In [47]:
start_word_id = context_tok_to_word_id[pair_scores_argmax // len(start_logits_context)]
end_word_id = context_tok_to_word_id[pair_scores_argmax % len(end_logits_context)]

Final answer:

In [48]:
predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id+1])
print("The answer to:\n" + my_question + "\nis:\n" + predicted_answer)

The answer to:
What are examples of economic actors?
is:
(worker, capitalist/business owner, landlord).


In [49]:
from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(my_context.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))